In [1]:
# Import basic libraries for the project

#Data modeling
import numpy as numpy
import pandas as pd
import matplotlib as plt
import sklearn

%matplotlib inline

# Modules for model
#from sklearn.


In [3]:
# Import training and validation data (PythonExercises\timeseries\notebooks)
df = pd.read_csv("./../../../data/bluebook-for-bulldozers/TrainAndValid.csv", low_memory = False)
df.head(5)

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000.0,999089,3157,121,3.0,2004,68.0,Low,11/16/2006 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000.0,117657,77,121,3.0,1996,4640.0,Low,3/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000.0,434808,7009,121,3.0,2001,2838.0,High,2/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500.0,1026470,332,121,3.0,2001,3486.0,High,5/19/2011 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000.0,1057373,17311,121,3.0,2007,722.0,Medium,7/23/2009 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# See the dtypes and check the existnace of null/missing values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 412698 entries, 0 to 412697
Data columns (total 53 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   SalesID                   412698 non-null  int64  
 1   SalePrice                 412698 non-null  float64
 2   MachineID                 412698 non-null  int64  
 3   ModelID                   412698 non-null  int64  
 4   datasource                412698 non-null  int64  
 5   auctioneerID              392562 non-null  float64
 6   YearMade                  412698 non-null  int64  
 7   MachineHoursCurrentMeter  147504 non-null  float64
 8   UsageBand                 73670 non-null   object 
 9   saledate                  412698 non-null  object 
 10  fiModelDesc               412698 non-null  object 
 11  fiBaseModel               412698 non-null  object 
 12  fiSecondaryDesc           271971 non-null  object 
 13  fiModelSeries             58667 non-null   o

### Since this is a time series problem, it is necessary to exact information from 'sales date' attribute
### Some of the values are in object type(we need to encode the data)
### There are entries with missing or NaN values

In [5]:
# make a copy of original data, so that we still keep the original
df_tmp = df.copy()


### Visualise the saledate format to confirm whether it needs any modification or not (parsing dates)

In [9]:
df_tmp['saledate']

0         11/16/2006 0:00
1          3/26/2004 0:00
2          2/26/2004 0:00
3          5/19/2011 0:00
4          7/23/2009 0:00
               ...       
412693      3/7/2012 0:00
412694     1/28/2012 0:00
412695     1/28/2012 0:00
412696      3/7/2012 0:00
412697     1/28/2012 0:00
Name: saledate, Length: 412698, dtype: object

In [12]:
# We need to change the format to YYYY-MM-DD
# Rearrange the data in an acesending order

# reimport the data using parse_date 
df = pd.read_csv("./../../../data/bluebook-for-bulldozers/TrainAndValid.csv", 
                 low_memory = False,
                 parse_dates=['saledate'])
# make a copy of original dataframe
df_tmp = df

# sort the dataframe in an acesending order by date
df_tmp.sort_values(by='saledate', inplace = True, ascending=True)
df_tmp.head(5)

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
205615,1646770,9500.0,1126363,8434,132,18.0,1974,NaN,NaN,1989-01-17,...,NaN,NaN,NaN,NaN,NaN,None or Unspecified,Straight,None or Unspecified,NaN,NaN
274835,1821514,14000.0,1194089,10150,132,99.0,1980,NaN,NaN,1989-01-31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
141296,1505138,50000.0,1473654,4139,132,99.0,1978,NaN,NaN,1989-01-31,...,NaN,NaN,NaN,NaN,NaN,None or Unspecified,Straight,None or Unspecified,NaN,NaN
212552,1671174,16000.0,1327630,8591,132,99.0,1980,NaN,NaN,1989-01-31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
62755,1329056,22000.0,1336053,4089,132,99.0,1984,NaN,NaN,1989-01-31,...,NaN,NaN,NaN,NaN,NaN,None or Unspecified,PAT,Lever,NaN,NaN


In [13]:
#date-time has a lot of attributes: It is good to perform feature engineering to group data based of year, month, date, or quater.
# Prediction may rely on feature engineered value instead of exact date

# Add datetime parameters for salesdate and we can drop salesdate column.
# Add year, month, day, start quater and year and end of quater and year
df_tmp['salesYear'] = df_tmp.saledate.dt.year
df_tmp['salesMonth'] = df_tmp.saledate.dt.month
df_tmp['salesDay'] = df_tmp.saledate.dt.day
df_tmp['salesQStart'] = df_tmp.saledate.dt.is_quarter_start
df_tmp['salesQEnd'] = df_tmp.saledate.dt.is_quarter_end
df_tmp['salesYStart'] = df_tmp.saledate.dt.is_year_start
df_tmp['salesYEnd'] = df_tmp.saledate.dt.is_year_end
df_tmp


,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Travel_Controls,Differential_Type,Steering_Controls,salesYear,salesMonth,salesDay,salesQStart,salesQEnd,salesYStart,salesYEnd
205615,1646770,9500.0,1126363,8434,132,18.0,1974,NaN,NaN,1989-01-17,...,None or Unspecified,NaN,NaN,1989,1,17,False,False,False,False
274835,1821514,14000.0,1194089,10150,132,99.0,1980,NaN,NaN,1989-01-31,...,NaN,Standard,Conventional,1989,1,31,False,False,False,False
141296,1505138,50000.0,1473654,4139,132,99.0,1978,NaN,NaN,1989-01-31,...,None or Unspecified,NaN,NaN,1989,1,31,False,False,False,False
212552,1671174,16000.0,1327630,8591,132,99.0,1980,NaN,NaN,1989-01-31,...,NaN,Standard,Conventional,1989,1,31,False,False,False,False
62755,1329056,22000.0,1336053,4089,132,99.0,1984,NaN,NaN,1989-01-31,...,Lever,NaN,NaN,1989,1,31,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
410879,6302984,16000.0,1915521,5266,149,99.0,2001,NaN,NaN,2012-04-28,...,None or Unspecified,NaN,NaN,2012,4,28,False,False,False,False
412476,6324811,6000.0,1919104,19330,149,99.0,2004,NaN,NaN,2012-04-28,...,NaN,NaN,NaN,2012,4,28,False,False,False,False
411927,6313029,16000.0,1918416,17244,149,99.0,2004,NaN,NaN,2012-04-28,...,NaN,NaN,NaN,2012,4,28,False,False,False,False
407124,6266251,55000.0,509560,3357,149,99.0,1993,NaN,NaN,2012-04-28,...,NaN,NaN,NaN,2012,4,28,False,False,False,False


In [14]:
# Now we can drop 'saledate' column

df_tmp.drop('saledate', axis=1, inplace=True)
df_tmp

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,...,Travel_Controls,Differential_Type,Steering_Controls,salesYear,salesMonth,salesDay,salesQStart,salesQEnd,salesYStart,salesYEnd
205615,1646770,9500.0,1126363,8434,132,18.0,1974,NaN,NaN,TD20,...,None or Unspecified,NaN,NaN,1989,1,17,False,False,False,False
274835,1821514,14000.0,1194089,10150,132,99.0,1980,NaN,NaN,A66,...,NaN,Standard,Conventional,1989,1,31,False,False,False,False
141296,1505138,50000.0,1473654,4139,132,99.0,1978,NaN,NaN,D7G,...,None or Unspecified,NaN,NaN,1989,1,31,False,False,False,False
212552,1671174,16000.0,1327630,8591,132,99.0,1980,NaN,NaN,A62,...,NaN,Standard,Conventional,1989,1,31,False,False,False,False
62755,1329056,22000.0,1336053,4089,132,99.0,1984,NaN,NaN,D3B,...,Lever,NaN,NaN,1989,1,31,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
410879,6302984,16000.0,1915521,5266,149,99.0,2001,NaN,NaN,D38E,...,None or Unspecified,NaN,NaN,2012,4,28,False,False,False,False
412476,6324811,6000.0,1919104,19330,149,99.0,2004,NaN,NaN,2064,...,NaN,NaN,NaN,2012,4,28,False,False,False,False
411927,6313029,16000.0,1918416,17244,149,99.0,2004,NaN,NaN,337G,...,NaN,NaN,NaN,2012,4,28,False,False,False,False
407124,6266251,55000.0,509560,3357,149,99.0,1993,NaN,NaN,12G,...,NaN,NaN,NaN,2012,4,28,False,False,False,False


In [15]:
df_tmp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 412698 entries, 205615 to 409203
Data columns (total 59 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   SalesID                   412698 non-null  int64  
 1   SalePrice                 412698 non-null  float64
 2   MachineID                 412698 non-null  int64  
 3   ModelID                   412698 non-null  int64  
 4   datasource                412698 non-null  int64  
 5   auctioneerID              392562 non-null  float64
 6   YearMade                  412698 non-null  int64  
 7   MachineHoursCurrentMeter  147504 non-null  float64
 8   UsageBand                 73670 non-null   object 
 9   fiModelDesc               412698 non-null  object 
 10  fiBaseModel               412698 non-null  object 
 11  fiSecondaryDesc           271971 non-null  object 
 12  fiModelSeries             58667 non-null   object 
 13  fiModelDescriptor         74816 non-null   o

### Now we need to get rid of the values that are in object type(we need to encode the data)
### i.e, turning data into numbers
### 1. string to numbers
### 2. bool to numbers

In [20]:
df.UsageBand.dtype

dtype('O')

In [22]:
pd.api.types.is_string_dtype(df_tmp['UsageBand'].unique())

False

In [18]:
# find the coloums with string values
# label =  attribute and content = value as per the documentation
str_obj = []
for label, content in df_tmp.items():
    if pd.api.types.is_string_dtype(content):
        str_obj.append(label)
        print(label)

str_obj

fiModelDesc
fiBaseModel
fiProductClassDesc
state
ProductGroup
ProductGroupDesc


['fiModelDesc',
 'fiBaseModel',
 'fiProductClassDesc',
 'state',
 'ProductGroup',
 'ProductGroupDesc']